In [ ]:
#To push the branch back to git you will probably need a personal access token
#https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token

#Before pushing to repo don't forget to right click the file on the left pane and git add

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [ ]:
#Download data
clientBQ = client(credentials=)
dataframe =clientBQ.query() #/ snowflake / SQL

#use this to train later on
#https://cloud.google.com/vertex-ai/docs/training/understanding-training-service
#Instead of using tfds.load you can download the dataset from anywhere else.

In [15]:
data, info = tfds.load(name='cifar10', as_supervised=True, with_info=True)
NUM_CLASSES = info.features['label'].num_classes
DATASET_SIZE = info.splits['train'].num_examples
print(DATASET_SIZE)

1000


In [16]:
def preprocess_data(image, label):
  image = tf.image.resize(image, (300,300))
  return tf.cast(image, tf.float32) / 255., label

In [17]:
# Create train/validation splits

# Shuffle dataset
dataset = data['train'].shuffle(1000)

train_split = 0.8
val_split = 0.2
train_size = int(train_split * DATASET_SIZE)
val_size = int(val_split * DATASET_SIZE)

train_data = dataset.take(train_size)
train_data  = train_data.map(preprocess_data)
train_data  = train_data.batch(64)

validation_data = dataset.skip(train_size)
validation_data  = validation_data.map(preprocess_data)
validation_data  = validation_data.batch(64)

In [18]:
feature_extractor_model = "inception_v3"

In [19]:
tf_hub_uri = f"https://tfhub.dev/google/imagenet/{feature_extractor_model}/feature_vector/5"


In [20]:
feature_extractor_layer = hub.KerasLayer(
    tf_hub_uri,
    trainable=False)

In [21]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(units=NUM_CLASSES)
])

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(train_data, validation_data=validation_data, epochs=1)

2022-09-12 08:24:21.613006: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


10/13 [======================>.......] - ETA: 21s - loss: 2.1005 - acc: 0.2922

2022-09-12 08:25:35.270372: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


13/13 [==============================] - ETA: 0s - loss: 2.0105 - acc: 0.3375

In [ ]:
#this should create a local folder with all the assets
#https://www.tensorflow.org/guide/keras/save_and_serialize
model.save("my_model")

In [ ]:
#https://cloud.google.com/storage/docs/uploading-objects
from google.cloud import storage

bucket_name = "automl-output-mlops"

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )
    
#TODO: fill in the arguments to use the funciton above
source_file = "my_model/saved_model.pb"
dest_blob = "saved_model.pb"
upload_blob(bucket_name, source_file, dest_blob)

In [ ]:
#https://cloud.google.com/vertex-ai/docs/model-registry/import-model#pre-built-container
#https://github.com/googleapis/python-aiplatform/blob/HEAD/samples/model-builder/upload_model_sample.py

from typing import Dict, Optional, Sequence

from google.cloud import aiplatform

PROJECT_ID = "mlops-demos-306914"

aiplatform.init(project=PROJECT_ID)
            
def upload_model_to_model_registry(
    project: str,
    display_name: str,
    serving_container_image_uri: str ="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest", # https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
    location: str = "us-central1",
    artifact_uri: Optional[str] = None,
    serving_container_predict_route: Optional[str] = None,
    serving_container_health_route: Optional[str] = None,
    description: Optional[str] = None,
    serving_container_command: Optional[Sequence[str]] = None,
    serving_container_args: Optional[Sequence[str]] = None,
    serving_container_environment_variables: Optional[Dict[str, str]] = None,
    serving_container_ports: Optional[Sequence[int]] = None,
    instance_schema_uri: Optional[str] = None,
    parameters_schema_uri: Optional[str] = None,
    prediction_schema_uri: Optional[str] = None,
    explanation_metadata: Optional[explain.ExplanationMetadata] = None,
    explanation_parameters: Optional[explain.ExplanationParameters] = None,
    sync: bool = True,
):

    aiplatform.init(project=project, location=location)

    model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        serving_container_predict_route=serving_container_predict_route,
        serving_container_health_route=serving_container_health_route,
        instance_schema_uri=instance_schema_uri,
        parameters_schema_uri=parameters_schema_uri,
        prediction_schema_uri=prediction_schema_uri,
        description=description,
        serving_container_command=serving_container_command,
        serving_container_args=serving_container_args,
        serving_container_environment_variables=serving_container_environment_variables,
        serving_container_ports=serving_container_ports,
        explanation_metadata=explanation_metadata,
        explanation_parameters=explanation_parameters,
        sync=sync,
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model

#TODO: fill in the arguments to use the funciton above
upload_model_to_model_registry(project=PROJECT_ID,display_name="my_display_name")

In [ ]:
#https://github.com/googleapis/python-aiplatform/blob/2bc9b2b0d048c29ba43c8b4c3ea51370515d08c3/samples/model-builder/create_batch_prediction_job_sample.py
from typing import Sequence, Union

from google.cloud import aiplatform

def create_batch_prediction_job_sample(
    project: str,
    location: str,
    model_resource_name: str,
    job_display_name: str,
    gcs_source: Union[str, Sequence[str]],
    gcs_destination: str,
    sync: bool = True,
):
    aiplatform.init(project=project, location=location)

    my_model = aiplatform.Model(model_resource_name)

    batch_prediction_job = my_model.batch_predict(
        job_display_name=job_display_name,
        gcs_source=gcs_source,
        gcs_destination_prefix=gcs_destination,
        sync=sync,
    )

    batch_prediction_job.wait()

    print(batch_prediction_job.display_name)
    print(batch_prediction_job.resource_name)
    print(batch_prediction_job.state)
    return batch_prediction_job

create_batch_prediction_job_sample()